In [7]:
import os, glob, random, math
from collections import Counter, defaultdict

import numpy as np
import pandas as pd

from PIL import Image, ImageOps
import matplotlib.pyplot as plt


In [8]:
INPUT_DIR = "/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes"

# List input datasets mounted in this notebook
for d in os.listdir(INPUT_DIR):
    print(d)

agri_data
classes.txt


In [9]:
# Quickly search for image files
img_exts = (".jpg",".jpeg",".png",".bmp")
all_imgs = []
for root, dirs, files in os.walk(INPUT_DIR):
    for f in files:
        if f.lower().endswith(img_exts):
            all_imgs.append(os.path.join(root, f))

print("Total images found:", len(all_imgs))
print("Example image path:", all_imgs[0] if all_imgs else None)

Total images found: 1300
Example image path: /kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_7209.jpeg
